In this Notebook:
- Handle Missing Values
- Encode Categorical Variables
- Create New Behavorial Features
- Output Clean Dataset for Modeling

In [2]:
import pandas as pd
ecom_df = pd.read_excel('../data/E Commerce Dataset.xlsx', 'E Comm')

In [3]:
# Handle Missing Values
ecom_df.isna().sum()

CustomerID                       0
Churn                            0
Tenure                         264
PreferredLoginDevice             0
CityTier                         0
WarehouseToHome                251
PreferredPaymentMode             0
Gender                           0
HourSpendOnApp                 255
NumberOfDeviceRegistered         0
PreferedOrderCat                 0
SatisfactionScore                0
MaritalStatus                    0
NumberOfAddress                  0
Complain                         0
OrderAmountHikeFromlastYear    265
CouponUsed                     256
OrderCount                     258
DaySinceLastOrder              307
CashbackAmount                   0
dtype: int64

In [4]:
# Numeric Features with Skewed Distribution should be filled with median
# Numeric Features with Normal Distribution should be filled with mean

# Tenure
# ecom_df['Tenure'].skew() # Positvely-Skewed (0.736)
ecom_df['Tenure'] = ecom_df['Tenure'].fillna(ecom_df['Tenure'].median())

# print(ecom_df)

# WarehouseToHome
# ecom_df['WarehouseToHome'].skew() # Positvely-Skewed (1.619)
ecom_df['WarehouseToHome'] = ecom_df['WarehouseToHome'].fillna(ecom_df['WarehouseToHome'].median())

# HourSpendOnApp
# ecom_df['HourSpendOnApp'].skew() # Normal-ish (-0.027)
ecom_df['HourSpendOnApp'] = ecom_df['HourSpendOnApp'].fillna(ecom_df['HourSpendOnApp'].mean())

# OrderAmountHikeFromLastYear
# ecom_df['OrderAmountHikeFromlastYear'].skew() # Positvely-Skewed (0.79)
ecom_df['OrderAmountHikeFromlastYear'] = ecom_df['OrderAmountHikeFromlastYear'].fillna(ecom_df['OrderAmountHikeFromlastYear'].median())

# CouponUsed 
# ecom_df['CouponUsed'].skew() # Positvely-Skewed (2.545)
ecom_df['CouponUsed'] = ecom_df['CouponUsed'].fillna(ecom_df['CouponUsed'].median())

# OrderCount 
# ecom_df['OrderCount'].skew() # Positvely-Skewed (2.196)
ecom_df['OrderCount'] = ecom_df['OrderCount'].fillna(ecom_df['OrderCount'].median())

# DaysSinceLastOrder -- Missing Data is meaningful. Must fill missing values while keeping meaningfulness
# ecom_df['DaySinceLastOrder'].skew() # Positvely-Skewed (1.191)
ecom_df['DaySinceLastOrder'] = ecom_df['DaySinceLastOrder'].fillna(999)

ecom_df.isna().sum()

CustomerID                     0
Churn                          0
Tenure                         0
PreferredLoginDevice           0
CityTier                       0
WarehouseToHome                0
PreferredPaymentMode           0
Gender                         0
HourSpendOnApp                 0
NumberOfDeviceRegistered       0
PreferedOrderCat               0
SatisfactionScore              0
MaritalStatus                  0
NumberOfAddress                0
Complain                       0
OrderAmountHikeFromlastYear    0
CouponUsed                     0
OrderCount                     0
DaySinceLastOrder              0
CashbackAmount                 0
dtype: int64

In [5]:
# Feature Engineering
import numpy as np

ecom_df['CashbackPerOrder'] = np.where(ecom_df['OrderCount'] == 0, 0, ecom_df['CashbackAmount']/ecom_df['OrderCount'])
ecom_df['RecentEngagementFlag'] = ecom_df['DaySinceLastOrder'].apply(lambda x: 1 if x <= 7 else 0)
ecom_df['AvgOrdersPerPaymentMode'] = ecom_df['PreferredPaymentMode'].map(ecom_df.groupby('PreferredPaymentMode')['OrderCount'].mean())
ecom_df['AvgOrdersPerMaritalStatus'] = ecom_df['MaritalStatus'].map(ecom_df.groupby('MaritalStatus')['OrderCount'].mean())

Engineered Features Explanations
1. CashbackPerOrder -> Normalizes total cashback by the number of orders to show per-order incentive strength. This helps identify customers who rely more on discounts, which may impact loyalty or retention.

2. RecentEngagementFlag -> Flags customers who have placed an order within 7 days since there last order. Recent engagement is a strong negative indicator of churn.

3. AvgOrdersPerPaymentMode -> Average amount of Orders per Payment Method. Helps uncover behavioral or loyalty trends across customer segments. For example, credit card users may order more frequently than wallet or cash users.

4. AvgOrdersPerMaritalStatus -> Average amount of Orders per Marital Status. Captures sociological patterns in shopping behavior and can help identify demographic-based loyalty differences. 

In [6]:
# Encode Categorical Features
# One-Hot Encoding
unordered_categoricals = ['PreferredLoginDevice','PreferredPaymentMode','Gender','PreferedOrderCat','MaritalStatus']
for col in unordered_categoricals:
    ecom_df[col] = ecom_df[col].astype(str)

ecom_df = pd.get_dummies(ecom_df,columns=unordered_categoricals,drop_first=False)
ecom_df.isna().sum()

CustomerID                               0
Churn                                    0
Tenure                                   0
CityTier                                 0
WarehouseToHome                          0
HourSpendOnApp                           0
NumberOfDeviceRegistered                 0
SatisfactionScore                        0
NumberOfAddress                          0
Complain                                 0
OrderAmountHikeFromlastYear              0
CouponUsed                               0
OrderCount                               0
DaySinceLastOrder                        0
CashbackAmount                           0
CashbackPerOrder                         0
RecentEngagementFlag                     0
AvgOrdersPerPaymentMode                  0
AvgOrdersPerMaritalStatus                0
PreferredLoginDevice_Computer            0
PreferredLoginDevice_Mobile Phone        0
PreferredLoginDevice_Phone               0
PreferredPaymentMode_CC                  0
PreferredPa

In [7]:
ecom_df.to_csv('../data/Cleaned_Ecom.csv',index=False)